# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import joblib
import requests
import json

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.core import Model
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.authentication import InteractiveLoginAuthentication

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'heart-failure'

experiment=Experiment(ws, experiment_name)

In [ ]:
# create a compute cluster
from azureml.core.compute import ComputeTarget, AmlCompute

cluster = "cluster00"

compute_target = ComputeTarget(workspace=ws, name=cluster)
if compute_target:
    print('Found existing cluster, use it.')
else:   
    print("No cluster with that name!")    

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

In [ ]:
found = False
key = "heart-failure-dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
dataset.take(5).to_pandas_dataframe()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)

# Specify parameter sampler
param_sampling = RandomParameterSampling({
        '--C' : choice(0.25,1),
        '--max_iter': choice(10,50,100)
    })

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Estimator and hyperdrive config
src = ScriptRunConfig(
    source_directory="./", 
    script="train.py",
    compute_target=cluster, 
    environment=sklearn_env
    )

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=early_termination_policy,
                                     run_config=src,
                                     max_total_runs=25,
                                     max_concurrent_runs=5)

In [ ]:
#Submit experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
print(hyperdrive_run.get_portal_url())
hyperdrive_run.get_details()

In [ ]:
hyperdrive_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_file_names = best_run.get_file_names()
parameter_values = best_run.get_details()['runDefinition']['arguments']


print('Best Run Id: ', best_run.id)
print('Best Run Metrics: ', best_run_metrics) 
print('Best Run Accuracy:', best_run_metrics['Accuracy']) 
print('Best Run File Names:', best_run_file_names) 

# print the model parameter values for the best run
print('\n Model parameters: ')
for elem in parameter_values:
    print(elem)

In [ ]:
best_model_deploy = best_run.register_model(model_name='hf-best-model', model_path='outputs/model')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = Model(ws, 'hf-best-model')
env = Environment.from_conda_specification(name='prediction-env', file_path='conda_dependencies.yml')

inference_config = InferenceConfig(entry_script='score.py', environment=env)

service_name = 'heart-failure-prediction'
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace = ws,
                       name = service_name,
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = aci_config)

In [ ]:
service.wait_for_deployment(show_output=True)
print(service.scoring_uri)
print(service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
auth = InteractiveLoginAuthentication()
header = auth.get_authentication_header()

data = {
  "data":
    {
      "age": 51, 
      "anaemia": 0, 
      "creatinine_phosphokinase": 600, 
      "diabetes": 1, 
      "ejection_fraction": 20, 
      "high_blood_pressure": 1, 
      "platelets": 265000, 
      "serum_creatinine": 1.9, 
      "serum_sodium": 130, 
      "sex": 0, 
      "smoking": 1,
      "time": 4
    },
  }

body = json.dumps(data)
response = requests.post(service.scoring_uri,
                         headers=header,
                         data=body)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

